<a href="https://colab.research.google.com/github/MarcelodeFreitas/udemy_deep_learning_pytorch_python/blob/main/Projeto_4_Classifica%C3%A7%C3%A3o_bin%C3%A1ria_breast_cancer_classificar_somente_um_registro_e_salvar_o_classificador.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto 4: Classificação binária brest cancer - classificar somente um registro e salvar o classificador

## Etapa 1: Importação das bibliotecas

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
from sklearn.metrics import confusion_matrix, accuracy_score
import torch
import torch.nn as nn
torch.__version__

'1.13.1+cpu'

## Etapa 2: Base de dados

In [5]:
np.random.seed(123)
torch.manual_seed(123)

In [6]:
previsores = pd.read_csv('./databases/entradas_breast.csv')
classe = pd.read_csv('./databases/saidas_breast.csv')

In [7]:
previsores = torch.tensor(np.array(previsores), dtype = torch.float)
classe = torch.tensor(np.array(classe), dtype = torch.float)

In [8]:
type(previsores)

torch.Tensor

## Etapa 3: Transformação dos dados para tensores


In [9]:
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(previsores, classe), batch_size = 10, shuffle = True)

## Etapa 4: Construção do modelo

In [10]:
class classificador_torch(nn.Module):
    def __init__(self):
        super().__init__()
        self.dense0 = nn.Linear(30, 8)
        # Mesmos parâmetors usados pelo kernel initializer do Keras [https://keras.io/initializers/, ver sessão RandomNormal]
        torch.nn.init.normal_(self.dense0.weight, mean = 0.0, std= 0.05)
        self.dense1 = nn.Linear(8, 8)
        torch.nn.init.normal_(self.dense1.weight, mean = 0.0, std= 0.05)
        self.dense2 = nn.Linear(8, 1)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        self.output = nn.Sigmoid()

    def forward(self, X):
        X = self.dense0(X)
        X = self.activation(X)
        X = self.dropout(X)
        X = self.dense1(X)
        X = self.activation(X)
        X = self.dropout(X)
        X = self.dense2(X)
        X = self.output(X)
        return X

In [11]:
classificador = classificador_torch()

In [12]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(classificador.parameters(), lr = 0.001,
                             weight_decay = 0.0001)

## Etapa 5: Treinamento do modelo

In [13]:
for epoch in range(100):
    running_loss = 0.

    for data in train_loader:
        inputs, labels = data
        optimizer.zero_grad()

        outputs = classificador(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print('Época %3d: perda %.5f' % (epoch+1, running_loss/len(train_loader)))

Época   1: perda 2.00713
Época   2: perda 0.74530
Época   3: perda 0.65829
Época   4: perda 0.63484
Época   5: perda 0.60440
Época   6: perda 0.58194
Época   7: perda 0.55260
Época   8: perda 0.51667
Época   9: perda 0.50661
Época  10: perda 0.49246
Época  11: perda 0.51614
Época  12: perda 0.45838
Época  13: perda 0.46064
Época  14: perda 0.46265
Época  15: perda 0.42716
Época  16: perda 0.44734
Época  17: perda 0.41308
Época  18: perda 0.41712
Época  19: perda 0.39283
Época  20: perda 0.38883
Época  21: perda 0.38020
Época  22: perda 0.38620
Época  23: perda 0.38924
Época  24: perda 0.34543
Época  25: perda 0.37637
Época  26: perda 0.34537
Época  27: perda 0.34335
Época  28: perda 0.32737
Época  29: perda 0.34000
Época  30: perda 0.32594
Época  31: perda 0.32785
Época  32: perda 0.32035
Época  33: perda 0.32407
Época  34: perda 0.33554
Época  35: perda 0.31132
Época  36: perda 0.28811
Época  37: perda 0.28319
Época  38: perda 0.27922
Época  39: perda 0.27540
Época  40: perda 0.26621


## Etapa 6: Classificar somente um registro

In [14]:
novo = torch.tensor([[15.80, 8.34, 118, 900, 0.10, 0.26, 0.08, 0.134, 0.178,
                  0.20, 0.05, 1098, 0.87, 4500, 145.2, 0.005, 0.04, 0.05, 0.015,
                  0.03, 0.007, 23.15, 16.64, 178.5, 2018, 0.14, 0.185,
                  0.84, 158, 0.363]], dtype = torch.float)

In [15]:
classificador.eval()

classificador_torch(
  (dense0): Linear(in_features=30, out_features=8, bias=True)
  (dense1): Linear(in_features=8, out_features=8, bias=True)
  (dense2): Linear(in_features=8, out_features=1, bias=True)
  (activation): ReLU()
  (dropout): Dropout(p=0.2, inplace=False)
  (output): Sigmoid()
)

In [16]:
previsao = classificador(novo)

In [17]:
previsao

tensor([[1.]], grad_fn=<SigmoidBackward0>)

In [18]:
previsao = previsao.detach()

In [19]:
previsao

tensor([[1.]])

In [20]:
previsao = previsao.numpy()

In [21]:
type(previsao)

numpy.ndarray

In [22]:
previsao

array([[1.]], dtype=float32)

In [23]:
previsao = (previsao > 0.5)
previsao

array([[ True]])

# Etapa 7: Salvar o classificador

In [24]:
# Quando salvar, você precisa chamar classificador.state_dict() (com os parêntese no final),
# ao invés de classificador.state_dict

In [25]:
classificador.state_dict()

OrderedDict([('dense0.weight',
              tensor([[-6.4070e-01, -1.2105e-02, -3.9366e-01, -1.0184e-01,  8.3805e-02,
                       -5.0670e-02,  5.2164e-02, -5.6332e-02,  4.0225e-02,  8.9479e-02,
                       -1.8339e-02,  2.4520e-03,  7.8185e-04, -1.7233e-02,  2.6877e-02,
                        2.3057e-01, -3.7470e-01, -3.0753e-01, -1.0798e-01,  4.3265e-01,
                       -4.8195e-01,  2.3206e-02, -3.2733e-01,  1.8466e-01,  3.0953e-02,
                        3.9079e-02,  2.1703e-02,  6.3821e-02,  1.7347e-02,  6.6465e-01],
                      [ 7.5206e-02,  1.4571e-02,  5.1368e-02, -5.0353e-02,  1.0085e-01,
                        3.4654e-02,  7.1073e-02, -1.4861e-03, -3.9469e-02, -3.5528e-02,
                        2.8993e-02, -2.2005e-03, -4.1547e-02,  7.9065e-02, -1.4188e-09,
                        2.0390e-06,  2.0629e-02, -6.4045e-09, -2.2665e-08,  2.7463e-09,
                       -8.1432e-02,  4.5196e-02, -4.9173e-02, -4.8898e-02, -7.0585e-02,


In [27]:
# Para salvar o classificador com a versão 1.5.0 do PyTorch, use o código abaixo

torch.save(classificador.state_dict(), 'checkpoint.pth')